In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Ridge Regression
class RidgeRegression() :
    def __init__( self, l2_penality, learning_rate = 0.001, iterations = 10000 ) :
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.l2_penality = l2_penality

    # Function for model training
    def fit( self, X, Y ) :
        # no_of_training_examples, no_of_features
        self.m, self.n = X.shape

        # weight initialization
        self.W = np.zeros( self.n )
        self.b = 0
        self.X = X
        self.Y = Y

        # gradient descent learning
        for i in range( self.iterations ) :
            self.update_weights()
        return self

    # Helper function to update weights in gradient descent
    def update_weights( self ) :
        Y_pred = self.predict( self.X )

        # calculate gradients
        dW = ( - ( 2 * ( self.X.T ).dot( self.Y - Y_pred ) ) +
               ( 2 * self.l2_penality * self.W ) ) / self.m
        db = - 2 * np.sum( self.Y - Y_pred ) / self.m

        # update weights
        self.W = self.W - self.learning_rate * dW
        self.b = self.b - self.learning_rate * db

        for i in range(13) :
          if self.W[i] <= 0 :
            self.W[i] = 0.001
          else :
            self.W[i] = self.W[i]





        return self

    # Hypothetical function  h( x )
    def predict( self, X ) :
        return X.dot( self.W ) + self.b

# Driver code
def main() :
    # Importing dataset
    df = pd.read_csv("//content/Data_for_model (2).csv")
    if "Unnamed: 0" in df.columns:
       df = df.drop(columns = "Unnamed: 0")
    df = df[df["shows"] != "naagin_6"].reset_index(drop = True)
    df = df[df["shows"] != "kuch_toh_hai_naagin_ek_naye_rang_mein"].reset_index(drop = True)

    df_ = df.drop(columns = ["shows","actual cume reach"])

    # Binary Columns
    def binary_columns(data) :
        binary_columns = []
        for column in data.columns :
          unique_values = df[column].dropna().unique()
          if len(unique_values) <= 2 or sorted(unique_values) == [0, 1]:
                binary_columns.append(column)
        return binary_columns

    binary_columns_list = list(binary_columns(df_))

    print(binary_columns_list)
    columns_to_standardize = [ 'Vertical Duplication', 'Previous show cume reach',
       'Overall Channel Pre 4 weeks Reach', 'Slot_Pre 4 Wks Reach', 'Lead out', 'GRPs', 'Home Promo Days', 'Network Promo Days', 'Outside Network Promo Days',
        'Non_tv_marketing', 'All_Views(social video/ott/GDN/sub_mediums/oth_glance_zapr)',
       'Own_Media_Reach', 'Pr_Reach']


    preprocessor = ColumnTransformer( transformers=[('standardize', StandardScaler(), columns_to_standardize), ('binary', 'passthrough', binary_columns)])

    # Creating the pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor)
    ])

    # Applying the pipeline to the dataframe
    processed_df = pipeline.fit_transform(df_)

    final_df = pd.DataFrame(processed_df, columns= list(columns_to_standardize) + list(binary_columns_list))
    for i in columns_to_standardize :
      for j in range(len(final_df)) :
        if final_df[i][j]>= 3:
         final_df[i][j] = 3

        elif final_df[i][j] <= -3:
          final_df[i][j] = -3

    X = final_df.values
    Y = df["actual cume reach"].values


    model = RidgeRegression(iterations = 100000, learning_rate = 0.001, l2_penality = 9)
    model.fit(X, Y)

    # Prediction on test set
    y_pred = model.predict(X)
    mape= mean_absolute_percentage_error(Y, y_pred)

    print("mape_test", mape)
    print("Trained b", round(model.b, 2))

    print("**********************************************")
    df_coef = pd.DataFrame({"features":["intercept"]+list(final_df.columns), "coef" :[round(model.b, 2)] +list(np.round(model.W, 5))})
    print(df_coef)
    df_coef.to_csv("coef_of_1st_stage.csv", index = False)
    print("************************************************")
    df_actual_v_pred = pd.DataFrame({"shows":df["shows"],"actual_cume_reach": Y, "predicted_cume_reach": y_pred})
    df_actual_v_pred.to_csv("1st_stage_model_results.csv")

    df.to_csv("data_for_second_stage.csv", index = False)

    y = df_actual_v_pred["actual_cume_reach"]
    y_pred = df_actual_v_pred["predicted_cume_reach"]
    shows = df_actual_v_pred["shows"]

    sorted_indices = np.argsort(y)
    y_sorted = np.array(y)[sorted_indices]
    y_pred_sorted = np.array(y_pred)[sorted_indices]
    shows_sorted = np.array(shows)[sorted_indices]

    # Plotting the sorted data
    plt.figure(figsize=(10, 6))
    plt.plot(shows_sorted, y_sorted, label='Actual Values', marker='o')
    plt.plot(shows_sorted, y_pred_sorted, label='Predicted Values', marker='x')

    # Adding title and labels
    plt.title('Actual vs Predicted Values (Sorted)')
    plt.xlabel('Index')
    plt.ylabel('Values')
    plt.xticks(rotation = 90)
    plt.legend()
    plt.show()


if __name__ == "__main__" :
    main()

['nf_to_f', 'Leads Scale 1', 'Leads Scale 2', 'Leads Scale 3', 'Genre Family Drama', 'Genre Mytho / Historical / Fantasy', 'Genre Romance', 'Genre Social Issue', 'Launchday Mon', 'Launchday Sat', 'Launchday Sun', 'Launchday Tue', 'Launchday Wed']
mape_test 0.06752612839796597
Trained b 5.56
**********************************************
                                             features     coef
0                                           intercept  5.56000
1                                Vertical Duplication  0.19958
2                            Previous show cume reach  0.37788
3                   Overall Channel Pre 4 weeks Reach  0.16669
4                                Slot_Pre 4 Wks Reach  0.29003
5                                            Lead out  0.04891
6                                                GRPs  0.15416
7                                     Home Promo Days  0.03256
8                                  Network Promo Days  0.13695
9                          Outs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Ridge Regression
class RidgeRegression:
    def __init__(self, l2_penalty, learning_rate=0.001, iterations=10000):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.l2_penalty = l2_penalty

    # Function for model training
    def fit(self, X, Y):
        # no_of_training_examples, no_of_features
        self.m, self.n = X.shape

        # weight initialization
        self.W = np.zeros(self.n)
        self.b = 0
        self.X = X
        self.Y = Y

        # gradient descent learning
        for i in range(self.iterations):
            self.update_weights()
        return self

    # Helper function to update weights in gradient descent
    def update_weights(self):
        Y_pred = self.predict(self.X)

        # calculate gradients
        dW = (-(2 * (self.X.T).dot(self.Y - Y_pred)) + (2 * self.l2_penalty * self.W)) / self.m
        db = -2 * np.sum(self.Y - Y_pred) / self.m

        # update weights
        self.W -= self.learning_rate * dW
        self.b -= self.learning_rate * db

        if self.W[2] <= 0:
          self.W[2] =0.001
        else :
          self.W[2] = self.W[2]


        if self.W[3] <= 0:
          self.W[3] =0.001
        else :
          self.W[3] = self.W[3]


        if self.W[4] <= 0:
          self.W[4] =0.001
        else :
          self.W[4] = self.W[4]


        return self

    # Hypothetical function  h(x)
    def predict(self, X):
        return X.dot(self.W) + self.b

def main():
    # Importing dataset
    df = pd.read_csv("/content/data_for_second_stage.csv")

    results_df = pd.read_csv("/content/1st_stage_model_results.csv")
    coef_df = pd.read_csv("coef_of_1st_stage.csv")

    if "Unnamed: 0" in df.columns:
        df = df.drop(columns="Unnamed: 0")

    if "Unnamed: 0" in results_df.columns:
        results_df = results_df.drop(columns="Unnamed: 0")

    if "Unnamed: 0" in coef_df.columns:
        coef_df = coef_df.drop(columns="Unnamed: 0")


    non_marketing_columns = ["shows", "Non_tv_marketing", "Print", "DTH", "Radio", "OOH", "OGA"]
    model_nontv_marketing = df[non_marketing_columns]

    scaler = StandardScaler()
    columns_to_scale = ["Non_tv_marketing", "Print", "DTH", "Radio", "OOH", "OGA"]
    model_nontv_marketing[columns_to_scale] = scaler.fit_transform(model_nontv_marketing[columns_to_scale])
    model_nontv_marketing[columns_to_scale] = model_nontv_marketing[columns_to_scale].clip(lower=-3, upper=3)

    model_nontv_marketing["pred_nontv_marketing"] = coef_df.loc[coef_df["features"] =="Non_tv_marketing", "coef"].values[0] * model_nontv_marketing["Non_tv_marketing"]
    merged_non_tv_marketing = pd.merge(model_nontv_marketing, results_df, on="shows")
    merged_non_tv_marketing["prediction_with_error"] = (merged_non_tv_marketing["pred_nontv_marketing"] + (merged_non_tv_marketing["actual_cume_reach"] - merged_non_tv_marketing["predicted_cume_reach"]))

    final_df = merged_non_tv_marketing[["Print", "DTH", "Radio", "OOH", "OGA"]]
    X = final_df.values
    Y = merged_non_tv_marketing["prediction_with_error"].values

    model = RidgeRegression(iterations=100000, learning_rate=0.001, l2_penalty=9)
    model.fit(X, Y)

    # Prediction on the training set
    y_pred = model.predict(X)
    df_coef = pd.DataFrame({"features": final_df.columns, "coef": np.round(model.W, 5)})
    coef_df = pd.concat([coef_df, df_coef])
    coef_df.to_csv("coef_of_two_stage_model.csv", index = False)


if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Ridge Regression
class RidgeRegression() :
    def __init__( self, l2_penality, learning_rate = 0.001, iterations = 10000 ) :
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.l2_penality = l2_penality

    # Function for model training
    def fit( self, X, Y ) :
        # no_of_training_examples, no_of_features
        self.m, self.n = X.shape

        # weight initialization
        self.W = np.zeros( self.n )
        self.b = 0
        self.X = X
        self.Y = Y

        # gradient descent learning
        for i in range( self.iterations ) :
            self.update_weights()
        return self

    # Helper function to update weights in gradient descent
    def update_weights( self ) :
        Y_pred = self.predict( self.X )

        # calculate gradients
        dW = ( - ( 2 * ( self.X.T ).dot( self.Y - Y_pred ) ) +
               ( 2 * self.l2_penality * self.W ) ) / self.m
        db = - 2 * np.sum( self.Y - Y_pred ) / self.m

        # update weights
        self.W = self.W - self.learning_rate * dW
        self.b = self.b - self.learning_rate * db



        if self.W[1] <= 0:
          self.W[1] = 0.001
        else :
          self.W[1] = self.W[1]


        if self.W[2] <= 0:
          self.W[2] =0.001
        else :
          self.W[2] = self.W[2]


        return self

    # Hypothetical function  h( x )
    def predict( self, X ) :
        return X.dot( self.W )


def main() :
    # Importing dataset
    df = pd.read_csv("/content/data_for_second_stage.csv")
    results_df = pd.read_csv("/content/1st_stage_model_results.csv")
    coef_df = pd.read_csv("coef_of_two_stage_model.csv")

    if "Unnamed: 0" in df.columns:
        df = df.drop(columns="Unnamed: 0")

    if "Unnamed: 0" in results_df.columns:
        results_df = results_df.drop(columns="Unnamed: 0")

    if "Unnamed: 0" in coef_df.columns:
        coef_df = coef_df.drop(columns="Unnamed: 0")

    marketing_columns = ["shows", 'GRPs',  'Home GRPs',  'Network GRPs',  'Outside Network GRPs']
    model_tv_marketing = df[marketing_columns]


    scaler = StandardScaler()
    columns_to_scale  = ['GRPs',  'Home GRPs',  'Network GRPs',  'Outside Network GRPs']
    model_tv_marketing[columns_to_scale] = scaler.fit_transform(model_tv_marketing[columns_to_scale])
    model_tv_marketing[columns_to_scale] = model_tv_marketing[columns_to_scale].clip(lower=-3, upper=3)

    model_tv_marketing["predicted_marketing"] = coef_df.loc[coef_df["features"] =="GRPs", "coef"].values[0] *model_tv_marketing["GRPs"]
    merged_tv_marketing  = pd.merge(model_tv_marketing, results_df, on="shows")
    merged_tv_marketing["prediction_with_error"] =  merged_tv_marketing["predicted_marketing"] + (merged_tv_marketing["actual_cume_reach"] - merged_tv_marketing["predicted_cume_reach"])

    final_df = merged_tv_marketing[['Home GRPs',  'Network GRPs',  'Outside Network GRPs']]
    X = final_df.values
    Y = merged_tv_marketing["prediction_with_error"].values
    model = RidgeRegression(iterations = 100000, learning_rate = 0.001, l2_penality = 9)
    model.fit(X, Y)

    # Prediction on test set
    y_pred = model.predict(X)
    mape= mean_absolute_percentage_error(Y, y_pred)
    df_coef = pd.DataFrame({"features":final_df.columns, "coef" :np.round(model.W, 5)})
    coef_df = pd.concat([coef_df, df_coef])
    coef_df.to_csv("coef_of_two_stage_model.csv", index = False)
    desired_features_column = list(df.columns)[1:]


    coefficients = pd.read_csv("coef_of_two_stage_model.csv").reset_index(drop=True)
    coef_feat = list(coefficients["features"])

    correct_index = {}
    for i in range(len(desired_features_column)) :
      for j in range(len(coef_feat)) :
        if coef_feat[j] == "intercept" :
          correct_index[coef_feat[j]] = coefficients["coef"][j]
        elif desired_features_column[i] == coefficients["features"][j] :
          correct_index[desired_features_column[i]] = coefficients["coef"][j]
        else :
          continue


    coef_two_stage = pd.DataFrame({"features": list(correct_index.keys()), "coef": list(correct_index.values())})
    coef_two_stage.to_csv("coef_of_two_stage_model.csv", index = False)
    print(coef_two_stage)

if __name__ == "__main__" :
    main()

In [ ]:
df = pd.read_csv("/content/model_analysis_colors (1).csv")
coefficients = pd.read_csv("coef_of_two_stage_model.csv").reset_index(drop=True)
desired_features_column = list(df.columns)[1:]
coef_feat = list(coefficients["features"])

correct_index = {}
for i in range(len(desired_features_column)) :
  for j in range(len(coef_feat)) :
    if coef_feat[j] == "intercept" :
      correct_index[coef_feat[j]] = coefficients["coef"][j]
    elif desired_features_column[i] == coefficients["features"][j] :
      correct_index[desired_features_column[i]] = coefficients["coef"][j]
    else :
      continue

coef_two_stage = pd.DataFrame({"features": list(correct_index.keys()), "coef": list(correct_index.values())})
coef_two_stage.to_csv("coef_of_two_stage_model_for_R.csv", index = False)

In [ ]:
coef_two_stage